In [ ]:
! pip install weaviate-client langchain langchain_community bitsandbytes accelerate pypdf cohere transformers torch 

In [ ]:
import weaviate
from langchain.retrievers.weaviate_hybrid_search import WeaviateHybridSearchRetriever
import torch
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline)
from langchain import HuggingFacePipeline
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank


In [ ]:
WEAVIATE_CLUSTER = "WEAVIATE_CLUSTER"
WEAVIATE_API_KEY = "WEAVIATE_API_KEY"
HF_TOKEN = "HUGGINGFACE_TOKEN"

In [ ]:
client = weaviate.Client(
    url=WEAVIATE_CLUSTER, auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY),
    additional_headers={
        "X-HuggingFace-Api-Key":HF_TOKEN
    } 
)

In [ ]:
client.is_ready()

In [ ]:
schema = {
    "classes": [
        {
            "class": "RAG",
            "description": "Documents for RAG",
            "vectorizer": "text2vec-huggingface",
            "moduleConfig": {"text2vec-huggingface": {"model": "sentence-transformers/all-MiniLM-L6-v2", "type":"text", "vectorizeClassName": True}},
            "properties": [
                {

                    "dataType": ["text"],
                    "description": "The content of the paragraph",
                    "moduleConfig": {
                        "text2vec-huggingface": {
                            "skip": False,
                            "vectorizePropertyName": False,

                        },

                    },

                    "name": "content",
                },
            ],
        },
    ]

}

In [ ]:
client.schema.create(schema)

In [ ]:
client.schema.get()

In [ ]:
retriever = WeaviateHybridSearchRetriever(
    client=client , # Keyword arguments to pass to the Weaviate client
    index_name="RAG", # The name of the index to use
    text_key="content",
    alpha=0.5,
    attributes=[],
    create_schema_if_missing=True,
    
)

In [ ]:
model_name = "HuggingFaceH4/zephyr-7b-beta"

In [ ]:
# Function for loading 4-bit quantized model
def load_quantized_model(model_name:str):

    """
    model_name: Name or path of the model to be loaded.
    return: Loaded quantized model.
    
    """

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        low_cpu_mem_usage=True
    )

    model = AutoModelForCausalLM.from_pretraind(
        model_name,
        torch_dtype = torch.bfloat16,
        quantization_config = bnb_confing
    )

    return model

In [ ]:
# Initialize_tokenizer
def initialize_tokenizer(model_name:str):
    """
    model_name: Name or path of the model for tokenizer initialization.
    return: Initialized tokenizer.

    """

    tokenizer = AutoTokenizer.from_pretrained(model_name, return_token_type_id=False)
    tokenizer.bos_token_id = 1 # Set beginning of sentence token id
    return tokenizer

In [ ]:
tokenizer = initialize_tokenizer(model_name=model_name)

In [ ]:
model = load_quantized_model(model_name=model_name)

In [ ]:
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    use_cache=True,
    device_map="auto",
    max_length=3000,
    do_sample=True,
    top_k = 5,
    #max_new_tokens=100,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id
)

In [ ]:
llm = HuggingFacePipeline(pipeline=pipeline)

In [ ]:
doc_path = "/content/Retrieval-Augmented-Generation-for-NLP.pdf"

In [ ]:
loader = PyPDFLoader(doc_path)

In [ ]:
docs = loader.load()

In [ ]:
retriever.add_documents(docs)

In [ ]:
retriever.invoke("What is RAG Token?",
                 score=True)

In [ ]:
hybrid_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

In [ ]:
result = hybrid_chain.invoke("What is RAG Token?")

In [4]:
COHERE_API_KEY = "COHERE_API_KEY"

In [ ]:
compressor = CohereRerank(cohere_api_key=COHERE_API_KEY)

In [ ]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [ ]:
compressod_docs = compression_retriever.get_relevant_documents("What is RAG Token?")

In [ ]:
print(compressod_docs[0])

In [ ]:
hybrid_chain_1 = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=compression_retriever
)

In [ ]:
response = hybrid_chain_1.invoke("What is Abstractive Question Answering?")